<hr style="height:2px;border:none" />
<h1><center> Engineering Tripos Part IIA Module Experiment: 4C7 </center></h1> 
<h1><center> Nonlinear Vibration, South Wing Mechanics Laboratory </center></h1>
<hr style="height:2px;border:none" />

## Objectives

-  To explore nonlinear effects due to high forcing amplitudes about resonance in mechanical structures.

- To construct a model to explain the nonlinear behaviour of the mechanism provided and use this model to simulate the behaviour of the system under specified conditions.

- Demonstrate use of the sonogram (time-varying spectrum) as an analytical tool to distinguish both frequency and temporal characteristics of a transient record.

## Introduction

In many instances a linear model is sufficient to approximate the behaviour of physical systems. However, in particular instances the linear model breaks down and a model with further complexity must be used. We will use the principles of nonlinear vibration theory described in the lectures to model the observed behaviour of a clamped-clamped beam subjected to high driving forces about the primary resonance.

This experiment is in two parts:

**Part A:** Measurement of the vibratory response of a clamped-clamped beam to variable drive forces.

**Part B:** Modelling and simulation of the nonlinear system in MATLAB.

## Report

Your report should describe the key results of the experiment, with plots when appropriate. A number of specific questions need to be addressed. The completed report should be securely tied together if it has loose pages, and attached to a completed coursework feedback coversheet. It should then be handed in via the white post-box outside room BE3-39 on the third floor of the Baker building or delivered to Hilde Fernandez in the Div C office before the final lecture of Michaelmas term.

## Apparatus

The apparatus comprises of a thin aluminium beam clamped at two ends to prevent axial displacement. Such a system exhibits a spring hardening effect (comment as to why this is the case in your report) with growing displacement amplitude for sufficiently large force inputs. A shaker is used to apply driving forces to the structure. The shaker derives its input from a PC-generated sine-wave (filtered by a low-pass filter) that can be varied in both frequency and amplitude. A National Instruments Data Acquisition Card (NI-DAQ) converts the specified signal to an analogue output, which is low-pass filtered, passed through channel 1 of an oscilloscope, and power-amplified by a current amplifier which provides the input to the shaker. An accelerometer is placed at the centre of the beam to measure the response of the structure to the applied drive input. The accelerometer output is coupled to an amplifier whose output is captured by the NI-DAQ using a Python package that this notebook uses and is described below. The PC will be used to drive the shaker and capture data from the accelerometer. Measurement data will be saved to your teaching system drive where it will be analysed and used for comparison with simulation. If you choose, you can analyse the data at any PC connected to the teaching system.

<hr style="height:2px;border:none" />

## Part A: Experimental procedure

### A1: Linear response

<hr style="height:2px;border:none" />

Start by importing the necessary modules (pydvma is a python package written for data acquisition at CUED).

Remember:

* to actually run a cell of code, click inside the cell then press 'shift+enter'
* the cell is running while [ * ] is displayed
* the cell has finished running when it changes to a number
* under the 'view' menu above you can toggle off the header and toolbar, which is useful to give a bit more screen space
* you can control the cell output view by clicking on the left beside the cell: single click to expand/compress, double-click to hide.

***

In [1]:
%gui qt

Wait a second before running the imports: it takes a moment for the previous command to take effect behind the scenes. If you get an error, just try it again.

In [ ]:
import matplotlib
import numpy as np
import pydvma as dvma

The first stage of the experiment is to measure the linear response due to a sweep input in force. This involves varying the force supplied by the shaker in frequency while fixing the amplitude at a sufficiently low forcing level to measure the linear response while sweeping upwards and downwards in frequency. Now open the logger window by running the next cell (shift+enter), with the following suggested settings:

* channels=2 (number of channels to record)
* fs=1500 (sampling rate in Hz)
* pretrig_samples=100 (starts logging 100 seconds before the signal starts being generated)
* stored_time=10 (time in seconds to store data)
* device_driver = 'nidaq'

In [ ]:
# acquisition setup
settings = dvma.MySettings(channels=2,
                           fs=1500,
                           stored_time=10,
                           pretrig_samples=100,
                           device_driver = 'nidaq')
logger = dvma.Logger(settings)

Familiarise yourself with the logger window:
- the centre panel contains the figure - note the zooming / panning tools under the figure.
- the left panel controls what is plotted (you can select which plot to show using the top dropdown)
- the right panel provides data analysis tools (you can select which tools are displayed using the top dropdown)
- the centre top panel is for basic data management (logging / loading / deleting)
- the centre bottom panel is to save data or figures.

Although the logger is set up to record data, we need to choose an output signal to generate. Use the ''Tool Selection'' dropdown at the top right to choose ''Generate Output''. Set the following options:

    Test name:    'up_0.2' (for example, noting that this will appear in the legend)
    Type:         'sweep'
    Amplitude:    0.2
    f1 (Hz):      40
    f2 (Hz):      60
    Duration (s): 9 (so that it finishes one second before logging finishes)

Press ''Preview Output'' to check it's what you expect to see. 

Press either ''Log with Output'' or simply ''Log Data'': the generated signal will be sent to the NI-DAQ card to output to the amplifier and the data from the input channels will be logged simultaneously.

You will then see the input and output signals displayed, including data for 1 s after the sweep excitation finished. A precaution: check that the data logging program has recorded the input signal from the accelerometer for the entire duration of the sweep.

__If the logger becomes unresponsive, or has output signal generation has stopped working, then you can restart this notebook by selecting 'Restart & Clear Output' from the 'Kernel' menu of the notebook.__

__Press ''Save Dataset'' and ''Save Figure'' to save your measurements and displayed figures to your directory for this experiment under your teaching system account. Choose a file name that describes this particular sweep: you will have saved several such files by the end of the experiment.__

To extract natural frequency and damping parameters for the system, run a fast Fourier transform (FFT) on the captured data by choosing ''Standard Tools'' from the tool selection dropdown (top right), then pressing ''Calculate FFT''. Then calculate the  transfer function by pressing ''Calculate TF'. The window and averaging options here should be set to ''None''.

Zoom in on the frequency range of interest (40 - 60 Hz) by entering the axis limits in the left panel or by using the zoom controls under the plot. The ''Auto Y'' button (left panel) can be used to set the amplitude range on the y-axis appropriately. 

Is the transfer function shaped as you would expect? Zoom in on a frequency range corresponding to amplitudes running from 5-10 dB on either side of the resonant frequency. In the left panel, use the plot type dropdown to choose a ''Nyquist'' plot. Then choose ''Mode Fitting'' from the tool selection dropdown (top right) which will fit a circle to the Nyquist plot. Extracted parameters such as the damping factor and resonant frequency will appear in the message area. (This method will be familiar to those also taking 4C6: the rest should just believe it!)

Now repeat the above procedure but sweep downwards in frequency by setting f1 to 60 Hz and the f2 to 40 Hz.

__Again save your data and figures in the same folder under a different filename. Do you notice any change in the measured parameters?__

<hr style="height:2px;border:none" />

### A2: Nonlinear response

<hr style="height:2px;border:none" />

Repeat the procedure outlined above performing upward and downward frequency sweeps for the forcing input for the same frequency range but for **different values of forcing amplitude**. The force amplitude is directly proportional to the amplitude of the sine wave generated by the computer: the actual voltage output is 5 times the normalised amplitude setting below. Use the ''Tool Selection'' dropdown at the top right to choose ''Generate Output''. Set the following options:

    Test name:    'up_0.2' (choose something systematic and compact: this will appear in the legend!)
    Type:         'sweep'
    Amplitude:    ... (x5 to give actual voltage)
    f min (Hz):   40
    f max (Hz):   60
    Duration (s): 9 (so that it finishes one second before logging finishes)

For each amplitude and sweep direction:

1. Press ''Delete All'' to clear the previous data.
- Change the output signal settings using the ''Generate Output'' tool - use the test name to keep track of the cases.
- Press ''Log Data''
- Press ''Calculate FFT'' and ''Calculate TF'' (no window, no averaging)
- Use the ''Mode Fitting'' tool, zoom into the peak, and press ''Fit'' to obtain the modal estimates (keep a note of these results)
- Press ''Save Dataset'' and ''Save Figure''
- Press ''Delete All'' and repeat

Run the experiment for sweep amplitudes equal to 3 V and 5 V (i.e. amplitude = 0.6 and 1, remember to change the test names accordingly to help you later). Especially for the runs with higher amplitude, you should hold the accelerometer wire gently away from the vibrating beam so that it does not create unwanted interference by buzzing. If something is wrong with one of the measurements, then press ''Delete Last'' and re-do the experiment, or ''Delete All'' and start again.

Note that each time you press ''Log Data'', the new measurement is added to the existing set of data: you need to **save each measurement separately**, using ''Delete All'' before logging the next test case. This lets you estimate the modal parameters for each test case independently (if you had logged all data into a single dataset the mode fitting tool would try to find a single frequency and damping factor for the whole set).

Once you have obtained an estimate of the modal parameters as a function of amplitude, then you can combine the datasets:

1. Press ''Delete All''
- Use ''Load Data'' repeatedly to combine all the sets of data (load the data in a sensible order so you know which set is which)
- Use the ''Figure Selection'' dropdown (top left) to choose which plot to view.
- Press ''Save Dataset'' to save the combined data, and ''Save Figure'' to save the plots you want to keep for your report.

__Don't forget to save your data and figures to the teaching system.__

Pay particular attention to the shape of the transfer function for each sweep. You can use the buttons in the left panel and legend lines to select particular data to show. Compare the upward and downward sweeps for the same amplitudes. Now compare the plots for varying amplitudes. Comment on your results.

__CHECK 1: Have you remembered to save your measurement datasets separately for each test case, with correct test names?__

__CHECK 2: Have you remembered to save the plots you need to keep for the report?__

__CHECK 3: Have you remembered to save the combined dataset of all test cases?__

<hr style="height:2px;border:none" />

## Part B: Modelling and Simulation

<hr style="height:2px;border:none" />

1. Python code is provided below that simulates the behaviour of a single-degree-of-freedom vibratory system with a Duffing nonlinearity. The required files can be accessed by typing ''edit solveduff'' and ''edit duffing'' in the Matlab Command Window. Save a copy of these files in your directory.

2. First, a model for the linear case (sweep amplitude = 1 V) will be developed. Let us describe the linear single-degree-of-freedom case by the standard equation: 
    equation
    
    In the above equation, the natural frequency p and damping coefficient  are given directly by the results of circle-fitting from your first test. Don’t forget that p is in radians/s. At a given point in a sweep, the forcing signal takes the form f (t) acos(t ) , where  represents an arbitrary phase of the input signal, a is the chosen amplitude in voltage units (1 V for the first experiment describing the linear case), and  is a scaling factor which describes the ratio of calibration factors for the two channels of measurement (acceleration and force). Using your values for natural frequency p and damping coefficient  , estimate the value of the scaling factor  from the peak output level from the time-series data in your first experiment (for which nonlinear terms can be neglected). Remember that your data is recording acceleration rather than displacement.
    
3. Next, to construct the non-linear model, we will examine the experimental data more carefully by conducting a “sonogram” analysis. A sonogram is a time-varying spectrum, illustrated below. A short section of the time history is copied and Fourier analysed — “Time window for FFT 1”. Then a section of the same length starting a little later in time is copied and analysed similarly (“FFT 2”), then another and another until a sufficient length of the whole time history has been covered.

    IMAGE

    By plotting in some suitable way this sequence of frequency spectra, we can now look at the response as a function of BOTH frequency and time (in much the same way that your brain processes sounds which you hear). To apply this to the logged experimental data: first load the MAT file; then select channel 2 only (accelerometer signal); then use “Options—Calculate sonogram”. Parameters for sonogram analysis are specified in a menu window. You are asked to provide values for three parameters:
    
    The values for items 1 and 3 are in digital samples, at 1500 per second: there is an option in the time-display window to show the time axis in samples rather than in seconds. Note that Number of spectra  Step size = total time range. Once the sonogram is calculated, you will see a contour plot on the screen. Time runs vertically up the screen, with “t = 0” at the bottom. Frequency appears along the horizontal axis. Contours of energy in the signal are plotted in this time/frequency plane. The contours are logarithmically spaced, initially at 5 dB intervals. You should experiment with the number of contours (suggested
    
    value = 8) and the spacing to obtain the clearest picture. Increase the numbers gradually from the
default values as it can take a long time to compute contours of small amplitude noisy signals, or
choose the ‘colourmap’ option at the bottom right of the screen which avoids this issue but gives
less clear plots. The frequency sweep should be clearly visible. You can adjust the sonogram
display using similar controls to the time display window: choose a range which brings out the
important features clearly, and stick to the same ranges for any cases you wish to plot and include in
your report, so that comparisons can be made easily.
You can also switch the display from a contour plot to a surface plot, which is better for getting a
general idea of the shape but less good for taking quantitative measurements. Using “Rotate3D”
from the “Tools” menu, you can use the mouse to rotate the surface to get the best view. The
“depth” of the surface plot is the same as the range specified for the contour plot.
4. Construct sonograms for each of the upward and downward sweeps for the measured data
How do the sonograms compare? Notice that for a cubic spring nonlinearity as indicated by the
Duffing equation, one should observe a third harmonic component on the sonogram plots with
increasing amplitude of forcing input. Does the third harmonic component increase with applied
input forcing? You will also notice that the sonograms for the measured data indicate the presence
of a second harmonic component in the output displacement. Comment upon the possible sources
for the appearance of the second harmonic in the measured data. Does the second harmonic
component increase with applied forcing amplitudes?
5. Motivated by the results of the sonogram analysis, the following equation is proposed to
model the non-linear single-degree-of-freedom vibratory system:
2 cos( ) 2 2 3 y py  p y y y a t 
Here, the parameters p ,  and  are the same as for the linear model. Form two non-dimensional
groups using the variables p, y,  , and  . Use this to estimate the values of  , and  for which
the effect of quadratic and cubic nonlinearities are expected to be comparable to the linear terms
(and remember that you are measuring acceleration, not displacement!). Given that in this
experiment the effect of nonlinearity is expected to be relatively weak, make a first guess of their
values. Do not be surprised if the numerical values for  and  seem rather extreme: try them and
see.
6. You are now set up to commence computer simulations of the non-linear system. Read the
commented routines “solveduff.m” and “duffing.m”. These are provided to you as a starting point
for your simulations. You should go through the code carefully to make sure that you fully
understand the syntax. The code simulates the force sweep that was conducted in the experiment for
a system described by a Duffing non-linearity. Modify your code to include a quadratic spring term
and incorporate the particular values of the extracted parameters into your model.
7. Run the code by typing “solveduff” at the command prompt. The output MAT files
generated by the simulation can be loaded using the program ‘log4c7’ by choosing “Read File”
from the Options menu. This will facilitate comparison of simulation results with experiment. You

will be able to run upward and downward sweeps as in your experiment. Run simulations for all values of input forcing that correspond to your experimental setup and save the data obtained for each of the cases for further analysis. You will also be able to run a sonogram analysis on the simulated data using the same procedure as for the experimental data.
8. Compare the simulation results carefully with your range of measurements, and iterate the parameter values of the simulation as necessary to obtain a best match. Your report should describe and illustrate how you carried out this process of obtaining best-fitting parameters: do not simply present the final numerical results. Comment on any discrepancies between simulation and experiment.
9. What other sources of nonlinearity might be operative in this experiment? Discuss briefly what characteristic effects you would expect these would produce, and whether any of these are strong candidates to account for the results of the experiment.
***